#### Création d'une base de données à partir d'un fichier csv contenant les informations sur les bénéficiaires effectifs

In [61]:
import psycopg2
import csv

In [62]:
# Connexion à la base de données PostgreSQL
conn = psycopg2.connect(
    dbname="rbe",
    user="postgres",
    password="postgres",
    host="localhost"
)

In [63]:
# Création d'un curseur pour exécuter des requêtes SQL
cur = conn.cursor()

In [64]:
# Création de la table des entreprises
cur.execute("""
    CREATE TABLE entreprises (
        siren VARCHAR(20) PRIMARY KEY
    )
""")

In [65]:
# Création de la table des personnes
cur.execute("""
    CREATE TABLE personnes (
        id SERIAL PRIMARY KEY,
        cle_unique VARCHAR(255),
        nom VARCHAR(255),
        prenoms VARCHAR(255),
        date_naissance VARCHAR(255),
        entreprise_siren VARCHAR(20) REFERENCES entreprises(siren)
    )
""")

In [66]:
# Lecture des données à partir du fichier CSV et insertion dans la base de données
with open("data/inpi-rbe.csv", "r") as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # Ignorer l'en-tête du CSV
    for row in csvreader:
        siren, nom, prenoms, date_naissance, cle_unique = row
        # Insérer les données dans la table entreprises
        cur.execute("INSERT INTO entreprises (siren) VALUES (%s) ON CONFLICT DO NOTHING", (siren,))
        # Insérer les données dans la table personnes
        cur.execute("INSERT INTO personnes (cle_unique, nom, prenoms, date_naissance, entreprise_siren) VALUES (%s, %s, %s, %s, %s)", (cle_unique, nom, prenoms, date_naissance, siren))


In [37]:
# Valider les modifications dans la base de données
conn.commit()

In [60]:
# Fermer le curseur et la connexion
cur.close()
conn.close()